# import, csv, json

In [45]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf   
## if error: python3.6 -m pip install scipy==1.2 --upgrade
import json 
import pprint
import re

In [46]:
df = pd.read_csv('data/NA/naclean.csv',low_memory=False)
print(df.shape)

(29473, 1186)


# regex + participants

In [47]:
def strippart(num):
    res = []
    for name in df.columns:
        if re.search('^participants_{}_'.format(num), name) != None:
            res += [name]
    res = [re.sub('^participants_{}_'.format(num), '', x) for x in res]
    return res

In [48]:
stripped = [set(strippart(x)) for x in range(0,10)]

In [49]:
[len(x) for x in stripped]

[109, 106, 106, 106, 108, 106, 105, 107, 107, 107]

In [50]:
common = list(set.intersection(*stripped))
len(common)

105

In [51]:
start = list(df.columns[:5])

team = []
for name in df.columns:
    if re.search('^team', name) != None:
        team += [name]

playercol = ['player_platformId', 'player_accountId', 'player_summonerName', 'player_summonerId', 
          'currentAccountId', 'player_matchHistoryUri', 'player_profileIcon']

partid = []
for name in df.columns:
    if re.search('^participantIdentities', name) != None:
        partid += [name]

def isolate(num):
    participant = ['participants_{}_{}'.format(num,x) for x in common]
    player = [x for x in partid if str(num) in x]
    res = df.copy()[start + team + participant + player]
    newcol = start + team + common + playercol
    res.columns = newcol
    res['participant_number'] = num
    return res

In [52]:
i = [isolate(x) for x in range(0,10)] 

In [53]:
participants = pd.concat(i)
participants.shape

(294730, 162)

In [54]:
participants

,gameId,gameCreation,gameDuration,queueId,gameVersion,teams_0_win,teams_0_firstBlood,teams_0_firstTower,teams_0_firstInhibitor,teams_0_firstBaron,...,stats_doubleKills,stats_perk3Var1,player_platformId,player_accountId,player_summonerName,player_summonerId,currentAccountId,player_matchHistoryUri,player_profileIcon,participant_number
0,3425927177,1589811429884,1548,420,10.10.320.3039,1,1,1,1,1,...,3,4590,NA1,HdSEE_2_wJd3FOo1t1GzHaqn_fEBA3ogvWV4XkHqhB5T4Q,Saiyans,slg_XD264Tf5OPLTFtklRkXBevhsx2M1soipEFA-NBTtDUU,HdSEE_2_wJd3FOo1t1GzHaqn_fEBA3ogvWV4XkHqhB5T4Q,/v1/stats/player_history/NA1/42407804,3002,0
1,3463315468,1592285561264,1499,420,10.12.324.5925,0,0,0,0,0,...,0,557,NA1,BUGcEEv-l4OFWCoKxcqTRiJm2P90ToFAs-jJNG5XXC63yI...,watch clannad,oBd4V3pA5V1hOBVyRDSj9OC_myutNzv-wyARZ5IyvFtnXyFI,BUGcEEv-l4OFWCoKxcqTRiJm2P90ToFAs-jJNG5XXC63yI...,/v1/stats/player_history/NA1/2121925252565312,23,0
2,3490775054,1594487307024,2300,420,10.14.327.8505,0,0,1,0,1,...,1,850,NA1,qg0OpXGX86H4EcgT8n66VueND9Ny1MZiweQwWz9VuZtRVp...,timal,Coeuu_2hrEWEJfgVd9ppv5sbZsH2B3coK4yl0aDPuHvWXpKV,qg0OpXGX86H4EcgT8n66VueND9Ny1MZiweQwWz9VuZtRVp...,/v1/stats/player_history/NA1/2252934392023072,4568,0
3,3449782290,1591330333433,1413,420,10.11.322.2991,0,1,0,0,0,...,1,2168,NA1,KLpPTF9mA2fmvDeV2YV3VPAHzetDhq-S5yruLosYLHL61M...,lMT Eika,7yVhyCE7BXDw_VVWxEsuQbgMj_GjsVg5_Duq6ZZA4zpF8sUE,KLpPTF9mA2fmvDeV2YV3VPAHzetDhq-S5yruLosYLHL61M...,/v1/stats/player_history/NA1/2378839681460352,3478,0
4,3482451986,1593821192664,1608,420,10.13.326.4870,1,1,0,1,1,...,0,382,NA1,wxbYpmkHf9DVZOBF_yhcoF2pPZ9cEkHyRG602B7wrTByC_...,malahuoguo fans,AjNo8fDBrQHVAmkcAjTVlsqBdOuaJJPlZuqnoyc4UT5P8RqD,wxbYpmkHf9DVZOBF_yhcoF2pPZ9cEkHyRG602B7wrTByC_...,/v1/stats/player_history/NA1/2471543443835776,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29468,3487563744,1594224435270,1549,420,10.14.327.1684,1,1,0,1,1,...,0,66,NA1,98ECGhIuA8Q6OuacZBoFiGHxP3Y21IcvVk8xlbDklXCzR9U,Dumahl,4BfwEx0sWzbooDAAlwSawkRSgw8nW2XxCPjs0W6oHOSYCEg,98ECGhIuA8Q6OuacZBoFiGHxP3Y21IcvVk8xlbDklXCzR9U,/v1/stats/player_history/NA1/236562517,3791,9
29469,3480158177,1593634490053,762,420,10.13.326.4870,1,0,1,1,0,...,0,0,NA1,WJEJ5P6XqTQakJNiRK6gz6I7X46w8DcbtkD2K71tPY1bdF...,James Jeong,IGC5QF4xMiAU8qqEIELHzsFhri0cy7GA7o8WuBzzOwvPjPg,WJEJ5P6XqTQakJNiRK6gz6I7X46w8DcbtkD2K71tPY1bdF...,/v1/stats/player_history/NA1/1999549387926176,3380,9
29470,3326738405,1584385654501,1407,420,10.5.312.392,1,1,1,1,1,...,0,24,NA1,iT86RgwVfyVpoVE87GJqeeEV9nr2esvQ0K5fLp7Gii0AqQ,ttvMemeLibrarian,_cL5jlhxAqL82PLXxh8S8VLaRptlpQyxeuI4XU81NCMUm3k,iT86RgwVfyVpoVE87GJqeeEV9nr2esvQ0K5fLp7Gii0AqQ,/v1/stats/player_history/NA1/50413575,12,9
29471,3479175144,1593561996811,2128,420,10.13.326.4870,1,0,0,1,0,...,0,736,0,HzNJg2zlU_YKZAzbLfzrLHbLYgAjL2kiEDdOyH0cRE78WA,Prabhably,nQZnVuE_FGMWS6QlfuPFKY1juDTedohndkmaBvvUm6OlTAU,HzNJg2zlU_YKZAzbLfzrLHbLYgAjL2kiEDdOyH0cRE78WA,/v1/stats/player_history/NA/36062060,22,9


In [55]:
participants.columns

Index(['gameId', 'gameCreation', 'gameDuration', 'queueId', 'gameVersion',
       'teams_0_win', 'teams_0_firstBlood', 'teams_0_firstTower',
       'teams_0_firstInhibitor', 'teams_0_firstBaron',
       ...
       'stats_doubleKills', 'stats_perk3Var1', 'player_platformId',
       'player_accountId', 'player_summonerName', 'player_summonerId',
       'currentAccountId', 'player_matchHistoryUri', 'player_profileIcon',
       'participant_number'],
      dtype='object', length=162)

In [56]:
compression_opts = dict(method='zip',
                        archive_name='naplayer.csv')  
participants.to_csv('naplayer.zip', index=False, compression=compression_opts)

# exploration

In [23]:
# Exploring columns
for name in df.columns:
    print(name)

gameId
gameCreation
gameDuration
queueId
gameVersion
teams_0_win
teams_0_firstBlood
teams_0_firstTower
teams_0_firstInhibitor
teams_0_firstBaron
teams_0_firstDragon
teams_0_firstRiftHerald
teams_0_towerKills
teams_0_inhibitorKills
teams_0_baronKills
teams_0_dragonKills
teams_0_riftHeraldKills
teams_0_bans_0_championId
teams_0_bans_0_pickTurn
teams_0_bans_1_championId
teams_0_bans_1_pickTurn
teams_0_bans_2_championId
teams_0_bans_2_pickTurn
teams_0_bans_3_championId
teams_0_bans_3_pickTurn
teams_0_bans_4_championId
teams_0_bans_4_pickTurn
teams_1_win
teams_1_firstBlood
teams_1_firstTower
teams_1_firstInhibitor
teams_1_firstBaron
teams_1_firstDragon
teams_1_firstRiftHerald
teams_1_towerKills
teams_1_inhibitorKills
teams_1_baronKills
teams_1_dragonKills
teams_1_riftHeraldKills
teams_1_bans_0_championId
teams_1_bans_0_pickTurn
teams_1_bans_1_championId
teams_1_bans_1_pickTurn
teams_1_bans_2_championId
teams_1_bans_2_pickTurn
teams_1_bans_3_championId
teams_1_bans_3_pickTurn
teams_1_bans_4_

In [24]:
### Participants 0 - 4 are on team 0
all(df["teams_0_win"] == df["participants_4_stats_win"])

True

In [25]:
champions = df[["teams_0_win", 
                "participants_0_championId", "participants_1_championId", 
                "participants_2_championId", "participants_3_championId", 
                "participants_4_championId", "participants_5_championId", 
                "participants_6_championId", "participants_7_championId",
                "participants_8_championId", "participants_9_championId",
                "participants_0_stats_win", "participants_1_stats_win", 
                "participants_2_stats_win", "participants_3_stats_win",
                "participants_4_stats_win", "participants_5_stats_win",
                "participants_6_stats_win", "participants_7_stats_win",
                "participants_8_stats_win","participants_9_stats_win",
                "participants_0_timeline_lane", "participants_1_timeline_lane", 
                "participants_2_timeline_lane", "participants_3_timeline_lane", 
                "participants_4_timeline_lane", "participants_5_timeline_lane", 
                "participants_6_timeline_lane", "participants_7_timeline_lane",
                "participants_8_timeline_lane", "participants_9_timeline_lane",]].copy()

In [26]:
champions

,teams_0_win,participants_0_championId,participants_1_championId,participants_2_championId,participants_3_championId,participants_4_championId,participants_5_championId,participants_6_championId,participants_7_championId,participants_8_championId,...,participants_0_timeline_lane,participants_1_timeline_lane,participants_2_timeline_lane,participants_3_timeline_lane,participants_4_timeline_lane,participants_5_timeline_lane,participants_6_timeline_lane,participants_7_timeline_lane,participants_8_timeline_lane,participants_9_timeline_lane
0,1,Leblanc,Hecarim,Ashe,Thresh,Rengar,Sett,Graves,Aatrox,Syndra,...,MIDDLE,JUNGLE,BOTTOM,BOTTOM,JUNGLE,MIDDLE,JUNGLE,TOP,MIDDLE,JUNGLE
1,1,Volibear,Caitlyn,Taliyah,Jayce,Leblanc,Swain,LeeSin,Zoe,Sett,...,JUNGLE,BOTTOM,BOTTOM,TOP,MIDDLE,BOTTOM,JUNGLE,MIDDLE,BOTTOM,TOP
2,1,Chogath,Ekko,RekSai,Aphelios,Sett,LeeSin,Taliyah,Alistar,Syndra,...,TOP,TOP,JUNGLE,BOTTOM,MIDDLE,JUNGLE,BOTTOM,BOTTOM,MIDDLE,TOP
3,0,Lux,Caitlyn,Elise,Galio,Qiyana,Graves,Pyke,Orianna,Camille,...,BOTTOM,BOTTOM,JUNGLE,TOP,MIDDLE,JUNGLE,BOTTOM,BOTTOM,TOP,MIDDLE
4,0,Xayah,Jayce,Sylas,Senna,Trundle,Maokai,Nocturne,Kaisa,Azir,...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27475,1,Sett,Kaisa,Jax,LeeSin,TwistedFate,Kassadin,Nidalee,Caitlyn,Camille,...,BOTTOM,BOTTOM,TOP,JUNGLE,MIDDLE,MIDDLE,JUNGLE,BOTTOM,MIDDLE,BOTTOM
27476,0,Olaf,Ashe,Sett,Zoe,Graves,Talon,Karthus,Volibear,Rakan,...,TOP,BOTTOM,BOTTOM,MIDDLE,JUNGLE,MIDDLE,JUNGLE,MIDDLE,BOTTOM,BOTTOM
27477,1,Lucian,Zac,Sylas,Soraka,Ezreal,Senna,Volibear,LeeSin,Aphelios,...,MIDDLE,JUNGLE,TOP,MIDDLE,BOTTOM,BOTTOM,TOP,JUNGLE,BOTTOM,MIDDLE
27478,1,Camille,Qiyana,Leona,Vayne,Karthus,Sett,Kaisa,Yasuo,Volibear,...,TOP,MIDDLE,BOTTOM,BOTTOM,JUNGLE,BOTTOM,BOTTOM,MIDDLE,TOP,JUNGLE


In [27]:
#series of total games per champ
champgames = []
for col in champions.columns:
    champgames.append(champions[col].value_counts())

totalgames = champgames[1]
for i in range(2,10):
    totalgames += champgames[i]

In [28]:
#series of win percentages per champ
champwins = {}
for i in range(0,5):
    champwins[i] = champions.groupby('participants_{}_championId'.format(i)
                                    ).mean()['teams_0_win']
for i in range(5,10):
    champwins[i] = 1 - champions.groupby('participants_{}_championId'.format(i)
                                        ).mean()['teams_0_win']    
totalwins = pd.DataFrame(champwins).mean(axis=1)

In [29]:
#create df
wins = totalwins.to_frame().join(totalgames.to_frame())
wins.columns = ['win_percent','num_games']
wins

,win_percent,num_games
876,0.470992,947
Aatrox,0.498911,1945
Ahri,0.540395,582
Akali,0.471929,2081
Alistar,0.511899,1357
...,...,...
Zed,0.485721,1662
Ziggs,0.475232,222
Zilean,0.487093,517
Zoe,0.504925,3468


In [38]:
import requests
import re

In [48]:
summoner = requests.get("http://ddragon.leagueoflegends.com/cdn/10.15.1/data/en_US/summoner.json").json()

In [49]:
items = requests.get("http://ddragon.leagueoflegends.com/cdn/10.15.1/data/en_US/item.json").json()

In [58]:
items['data']['1011']

{'name': "Giant's Belt",
 'description': '<stats>+380 Health</stats>',
 'colloq': ';',
 'plaintext': 'Greatly increases Health',
 'from': ['1028'],
 'into': ['3083', '3084', '3022', '3143', '3742'],
 'image': {'full': '1011.png',
  'sprite': 'item0.png',
  'group': 'item',
  'x': 144,
  'y': 0,
  'w': 48,
  'h': 48},
 'gold': {'base': 600, 'purchasable': True, 'total': 1000, 'sell': 700},
 'tags': ['Health'],
 'maps': {'11': True, '12': True, '21': True, '22': False},
 'stats': {'FlatHPPoolMod': 380},
 'depth': 2}

In [45]:
participants[[col for col in participants.columns if 'perk' in col]]

,stats_perk1Var1,stats_perk4Var3,stats_perk2,stats_perk0Var2,stats_perk0Var1,stats_perk3Var2,stats_perk1Var2,stats_perk2Var1,stats_perk4Var1,stats_perk4Var2,...,stats_perk5Var2,stats_perk3Var1,stats_perk3Var3,stats_perk2Var2,stats_perk4,stats_perk0Var3,stats_perk2Var3,stats_perkPrimaryStyle,stats_perk5Var1,stats_perkSubStyle
0,1011,0,8138,0,1848,5,0,30,1258,480,...,0,4590,0,0,9111,0,0,8100,921,8000
1,674,0,9105,616,1088,0,120,17,502,0,...,0,557,0,30,8473,471,0,8000,0,8400
2,1089,0,9105,0,817,0,300,15,5736,5,...,0,850,0,30,8135,0,0,8000,510,8100
3,793,0,8138,0,1033,4,0,18,250,828,...,0,2168,0,0,8226,0,0,8100,0,8200
4,250,0,8210,0,17,0,1060,16,756,0,...,4,382,0,0,8139,0,0,8200,4425,8100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29468,0,0,9104,0,145,0,0,0,3,0,...,1410,66,0,0,8345,0,0,8000,230,8300
29469,0,0,8444,0,84,0,0,147,0,0,...,0,0,0,0,8313,0,0,8400,0,8300
29470,250,0,8233,1721,720,0,321,6,436,0,...,478,24,0,50,8463,0,0,8200,697,8400
29471,250,0,8210,0,2256,0,486,0,3,0,...,0,736,0,0,8345,0,0,8200,0,8300


# fit

In [ ]:
#results = smf.logit('teams_0_win ~ participantIdentities_1_player_accountId', data=df).fit()
#results.summary()